# How to add new bundles into pyAFQ (SLF 1/2/3 Example)

pyAFQ is designed to be customizable and extensible. This example shows how you
can customize it to define a new bundle based on a definition of waypoint and
endpoint ROIs of your design.

In this case, we add sub-bundles of the superior longitudinal fasciculus,
based on work by Sami et al.


In [ ]:
from paths import afq_home

In [ ]:
import os.path as op
import plotly
import numpy as np
from IPython.display import Image

from AFQ.api.group import GroupAFQ
import AFQ.api.bundle_dict as abd
import AFQ.data.fetch as afd
from AFQ.definitions.image import ImageFile, RoiImage
import wget
import os
np.random.seed(1234)

## Get dMRI data

We will analyze twelve subjects from the Healthy Brain Network Processed Open
Diffusion Derivatives dataset (HBN-POD2). The data gets organized into a BIDS-compatible format.


In [ ]:
_, study_dir = afd.fetch_hbn_preproc(['NDARAA948VFH'])

## Define custom `BundleDict` object

We define the bundles with include/exclude ROIs and specific cleaning parameters.


In [ ]:
template_dir = op.join(afq_home, 'SLF_ROIs')
templates_afq = afd.read_templates(as_img=False)

# You can provide paths to the ROIs in the "include" and "exclude" keys of the bundle definition
bundles = abd.BundleDict({
    "L_SLF1": {
        "include": [template_dir + '/SFgL.nii.gz', template_dir + '/PaL.nii.gz'],
        "exclude": [templates_afq['SLFt_roi2_L']],
        "cross_midline": False,
        "mahal": {"clean_rounds": 20, "length_threshold": 4, "distance_threshold": 2}
    }
})

## Define GroupAFQ object

We configure the AFQ pipeline with predefined parameters and start the process.


In [ ]:
brain_mask_definition = ImageFile(
    suffix="mask",
    filters={'desc': 'brain', 'space': 'T1w', 'scope': 'qsiprep'}
)

my_afq = GroupAFQ(
    bids_path=study_dir,
    preproc_pipeline="qsiprep",
    participant_labels=["NDARAA948VFH"],
    output_dir=op.join(study_dir, "derivatives", "afq_slf"),
    brain_mask_definition=brain_mask_definition,
    tracking_params={"n_seeds": 4, "directions": "prob", "odf_model": "CSD", "seed_mask": RoiImage()},
    segmentation_params={"parallel_segmentation": {"engine": "serial"}},
    bundle_info=bundles
)

my_afq.export("profiles")

## Visualize a montage

We create montage visualizations of the SLF bundles.


In [ ]:
montage = my_afq.group_montage("L_SLF1", (3, 4), "Sagittal", "left", slice_pos=0.5)
Image(filename=montage[0])

## Interactive bundle visualization

We export individual bundle figures as interactive HTML files.


In [ ]:
bundle_html = my_afq.export("all_bundles_figure")
plotly.io.show(bundle_html["NDARAA948VFH"][0])

## References

- Romi Sagi, J.S.H. Taylor, Kyriaki Neophytou, et al. White matter associations with spelling performance.
- Alexander LM, Escalera J, Ai L, et al. An open resource for transdiagnostic research in pediatric mental health and learning disorders. Sci Data. 2017.
- Richie-Halford A, Cieslak M, Ai L, et al. An analysis-ready and quality-controlled resource for pediatric brain white-matter research. Scientific Data. 2022.
- Cieslak M, Cook PA, He X, et al. QSIPrep: an integrative platform for preprocessing and reconstructing diffusion MRI data. Nat Methods. 2021.
